# convect each results into excel

In [2]:
import pandas as pd

df = pd.read_csv('results/summarized_results.csv')

# Sort the dataframe

df_sorted = df.drop(columns=['nn_model', 'number_of_clients'])
# Sort the dataframe by skew, dataset, and heterogeneity_class

# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
# Add a newline before each occurrence of 'fedavg' in the 'exp_type' column
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
# Sort the dataframe by the defined order
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('results.xlsx', index=False)

In [5]:
df=pd.read_csv('granular_results/noqs.csv')
# Define the order and bold the 'cornflqs' exp_type
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('noqs.xlsx', index=False)

In [7]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/qs1.csv')
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs1.xlsx', index=False)


In [6]:
df=pd.read_csv('granular_results/qs2.csv')
# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted = df
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs2.xlsx', index=False)

# calculate ranking 

In [ ]:
def algo_ranking(dataset=None, skew=None, heterogeneity_class=None):
    import pandas as pd
    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv('results/summarized_results.csv')
    df.fillna('None', inplace=True)
    df = df[df['exp_type'] != 'oracle-centralized']
    df['exp_type'] = df['exp_type'] + '_' + df['params']
    
    # Filter the dataframe based on the input parameters
    if dataset:
        df = df[df['dataset'] == dataset]
    if skew:
        if skew == 'None':
            df = df[df['skew'] == skew]
        elif skew == 'quantity-skew':
            df = df[df['skew'] != 'None']
        else:
            df = df[df['skew'] == skew]
    if heterogeneity_class:
        df = df[df['heterogeneity_class'] == heterogeneity_class]

    # Step 2: Clean the data
    # Remove the '\pm' and uncertainty values from the 'accuracy' column
    df['accuracy'] = df['accuracy'].str.split(' ').str[0].astype(float)
    # Replace 'n/a' with NaN for missing values

    # Convert metric columns to numeric
    metrics = ['accuracy', 'ARI', 'AMI', 'hom', 'cmplt', 'vm']
    df[metrics] = df[metrics].apply(pd.to_numeric, errors='coerce')

    # Concatenate 'dataset', 'heterogeneity_class', and 'skew' into one column named 'group'
    df['group'] = df['dataset'] + '_' + df['heterogeneity_class'] + '_' + df['skew'] + '_' + df['seed']
    # Drop the original 'dataset', 'heterogeneity_class', and 'skew' columns
    df = df.drop(columns=['dataset', 'heterogeneity_class', 'skew', 'seed'])

    # Step 3: Group the data by the new 'group' column
    grouped = df.groupby('group')

    # Step 4: Rank algorithms within each group for each metric
    def rank_algorithms(group):
        for metric in metrics:
            group[f'{metric}_rank'] = group[metric].rank(ascending=False, method='min')
        return group

    ranked_groups = grouped.apply(rank_algorithms)
    best = ranked_groups.groupby('exp_type')[[f'{metric}_rank' for metric in metrics]].mean()
    best = best.sort_values(by='accuracy_rank')

    # Calculate the average accuracy for each exp_type
    avg_accuracy = ranked_groups.groupby('exp_type')['accuracy'].mean()
    best['avg_accuracy'] = avg_accuracy

    return best


In [2]:
algo_ranking()

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
ifca_best,8.171429,2.400000,2.371429,2.390476,1.523810,2.371429,80.544000
fedgroup_edc,10.122642,5.226415,5.377358,5.245283,6.037736,5.433962,79.481226
hcfl_euclidean,10.745455,6.372727,6.327273,6.200000,6.663636,6.254545,77.798455
cfl_None,17.545455,14.054545,14.081818,13.918182,14.172727,14.127273,72.230727
srfca_None,18.857143,4.342857,4.295238,4.600000,1.419048,4.533333,76.403810
fedprox_0.1,26.463636,NaN,NaN,NaN,NaN,NaN,58.294455
fedavg_None,31.472727,NaN,NaN,NaN,NaN,NaN,55.406273


In [3]:
algo_ranking(skew='quantity-skew-type-1')

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
ifca_best,6.066667,1.533333,1.466667,1.488889,1.444444,1.466667,79.542222
fedgroup_edc,11.688889,9.577778,9.933333,9.800000,10.711111,10.088889,76.554667
hcfl_euclidean,13.800000,13.244444,13.133333,13.044444,13.400000,12.955556,72.268000
srfca_None,14.377778,2.600000,2.600000,2.644444,1.533333,2.600000,75.737111
cfl_None,19.777778,18.111111,18.000000,18.088889,16.955556,17.955556,69.553556
fedprox_0.1,27.222222,NaN,NaN,NaN,NaN,NaN,58.138222
fedavg_None,32.733333,NaN,NaN,NaN,NaN,NaN,54.581778


In [4]:
algo_ranking(skew='quantity-skew-type-2')

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
hcfl_euclidean,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,86.485000
fedgroup_edc,7.812500,2.375000,2.375000,2.375000,3.312500,2.375000,87.537500
ifca_best,9.933333,4.466667,4.466667,4.466667,3.333333,4.466667,86.966667
srfca_None,17.066667,6.666667,6.333333,6.666667,1.000000,6.333333,84.448000
cfl_None,20.850000,18.400000,18.700000,18.250000,18.900000,18.800000,68.503000
fedprox_0.1,24.150000,NaN,NaN,NaN,NaN,NaN,56.117500
fedavg_None,28.500000,NaN,NaN,NaN,NaN,NaN,53.150000


In [5]:
algo_ranking(skew='quantity-skew')

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
ifca_best,7.033333,2.266667,2.216667,2.233333,1.916667,2.216667,81.398333
fedgroup_edc,10.672131,7.688525,7.950820,7.852459,8.770492,8.065574,79.435410
hcfl_euclidean,11.400000,9.476923,9.400000,9.338462,9.584615,9.276923,76.642462
srfca_None,15.050000,3.616667,3.533333,3.650000,1.400000,3.533333,77.914833
cfl_None,20.107692,18.200000,18.215385,18.138462,17.553846,18.215385,69.230308
fedprox_0.1,26.276923,NaN,NaN,NaN,NaN,NaN,57.516462
fedavg_None,31.430769,NaN,NaN,NaN,NaN,NaN,54.141231


In [6]:
algo_ranking(skew='None')

,accuracy_rank,ARI_rank,AMI_rank,hom_rank,cmplt_rank,vm_rank,avg_accuracy
exp_type,,,,,,,
fedgroup_edc,9.377778,1.888889,1.888889,1.711111,2.333333,1.866667,79.543333
ifca_best,9.688889,2.577778,2.577778,2.600000,1.000000,2.577778,79.404889
hcfl_euclidean,9.800000,1.888889,1.888889,1.666667,2.444444,1.888889,79.468222
cfl_None,13.844444,8.066667,8.111111,7.822222,9.288889,8.222222,76.564667
srfca_None,23.933333,5.311111,5.311111,5.866667,1.444444,5.866667,74.389111
fedprox_0.1,26.733333,NaN,NaN,NaN,NaN,NaN,59.418222
fedavg_None,31.533333,NaN,NaN,NaN,NaN,NaN,57.233556


# EXP CSV generation

In [ ]:
import pandas as pd
import itertools

# Define the list of tuples for exp_type and params
exp_params_list = [
    ('fedavg', 'None'),
    ('fedprox', 0.1),
    ('cfl', 'None'),
    ('hcfl', 'euclidean'),
    ('fedgroup', 'edc'),
    ('ifca', 'best'),
    ('srfca', 'None'),
    ('cornflqs', '(1,10,10)'),
    ('cornflqs', '(10,1,10)'),
    ('cornflqs', '(1,1,20)'),
    ('oracle-centralized', 'clustering')
]

# Define the base data
base_data = [
    ['mnist', 'concept-shift-on-features', 'None'],
    ['mnist', 'concept-shift-on-labels', 'None'],
    ['mnist', 'features-distribution-skew', 'None'],
    ['fashion-mnist', 'concept-shift-on-features', 'None'],
    ['fashion-mnist', 'concept-shift-on-labels', 'None'],
    ['fashion-mnist', 'features-distribution-skew', 'None'],
    ['kmnist', 'concept-shift-on-features', 'None'],
    ['kmnist', 'concept-shift-on-labels', 'None'],
    ['kmnist', 'features-distribution-skew', 'None'],
    ['mnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['mnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['mnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['fashion-mnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['fashion-mnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['fashion-mnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['kmnist', 'concept-shift-on-features', 'quantity-skew-type-1'],
    ['kmnist', 'concept-shift-on-labels', 'quantity-skew-type-1'],
    ['kmnist', 'features-distribution-skew', 'quantity-skew-type-1'],
    ['mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['fashion-mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['kmnist', 'features-distribution-skew', 'quantity-skew-type-2']
]

# Define fixed parameters
nn_model = 'linear'
number_of_clients = 100
num_samples_by_label = 50
num_clusters = 4
default_epochs = 10
rounds = 20
# Define the seed values
seed_values = [42, 43, 44, 45, 46]

# Create a list to store all the combinations
all_combinations = []

# Generate all combinations of base_data, exp_params_list, and seed_values
for base, (exp_type, params), seed in itertools.product(base_data, exp_params_list, seed_values):
    epochs = 100 if exp_type == 'oracle-centralized' else default_epochs
    combination = base + [exp_type, params, seed, nn_model, number_of_clients, num_samples_by_label, num_clusters, epochs, rounds]
    all_combinations.append(combination)

# Convert the list of combinations to a DataFrame
columns = ['dataset', 'heterogeneity_class', 'skew', 'exp_type', 'params', 'seed', 'nn_model', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds']
df_combinations = pd.DataFrame(all_combinations, columns=columns)

# Reorder the columns as specified
df_combinations = df_combinations[['exp_type', 'params', 'dataset', 'nn_model', 'heterogeneity_class', 'skew', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds', 'seed']]

# Save the DataFrame to a CSV file
df_combinations.to_csv('exp_configs.csv', index=False)